# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXDI1","BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [7]:
da.columns

Index(['BPXDI1', 'BPXSY1', 'RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'DMDEDUC2',
       'BMXBMI', 'SMQ020', 'SDMVSTRA', 'SDMVPSU', 'group'],
      dtype='object')

In [9]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

The ICC for diastolic blood pressure is 0.031, which is very close to the ICC for systolic blood pressure which we observed in a previous notebook to be 0.030. This is a small ICC but not insignificant, it proves that both blood pressure measurements are correlated within our clusters.

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [10]:
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDR + RIDAGEYR + BMXBMI", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC for this model is 0.030 which is very similar to the model fit in the previous question. This shows that even when we add more explanatory variables to our model, the correlation of diastolic blood pressure within our clusters remains relatively unchanged.

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [14]:
da["RIAGENDR"] = da["RIAGENDR"].replace({1: "Male", 2: "Female"})
male_da = da.loc[da["RIAGENDR"] == "Male"]
female_da = da.loc[da["RIAGENDR"] == "Female"]

# female model
model1 = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=male_da)
result1 = model1.fit()

# female model
model2 = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=female_da)
result2 = model2.fit()

x = pd.DataFrame({"male_params": result1.params, "male_SE": result1.bse,
                  "female_params": result2.params, "female_SE": result2.bse})
x = x[["male_params", "male_SE", "female_params", "female_SE"]]
print(x)
print('\n')
print('Male model ICC')
print(result1.cov_struct.summary())
print('\n')
print('Female model ICC')
print(result2.cov_struct.summary())

           male_params   male_SE  female_params  female_SE
Intercept    74.601880  0.982582      69.615209   0.957849
RIDAGEYR     -0.067271  0.017352      -0.017276   0.019320


Male model ICC
The correlation between two observations in the same cluster is 0.032


Female model ICC
The correlation between two observations in the same cluster is 0.028


__Q3a.__ What do you learn by comparing these two fitted models?

We see that the ICC for the two models differ. This means that gender is a variable that helps explain some of the intercluster dependency that we see in our data when .

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [15]:
# Fit a multilevel (mixed effects) model to handle dependent data
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI + C(RIDRETH1)",
           groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:              MixedLM Dependent Variable: BPXDI1     
No. Observations:   5102    Method:             REML       
No. Groups:         30      Scale:              153.0915   
Min. group size:    106     Likelihood:         -20100.7500
Max. group size:    226     Converged:          Yes        
Mean group size:    170.1                                  
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        63.218    1.117 56.595 0.000 61.029 65.407
RIAGENDR[T.Male]  2.750    0.349  7.883 0.000  2.066  3.433
C(RIDRETH1)[T.2]  1.505    0.697  2.160 0.031  0.139  2.871
C(RIDRETH1)[T.3]  0.512    0.597  0.858 0.391 -0.658  1.682
C(RIDRETH1)[T.4]  2.537    0.659  3.852 0.000  1.246  3.828
C(RIDRETH1)[T.5]  3.491    0.690  5.061 0.000  2.139  4.843
RIDAGEYR         -0.036    0.010 -3.590 0.000 -0.056 -0.016
BMXBMI            0.197    0.026  7.642 0.000  0.147  0.248
group Var         4.196    0.110                           
===========================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The variance for the groups estimated to be 4.196. This means that if we were to choose two groups at random, their random effects would differ on average by around 2.89 (this is calculated as the square root of 2*4.196). This means that there is significant differences between the slopes and intercepts of our groups.

In [20]:
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI + C(RIDRETH1)", 
                                groups="group", re_formula = '1 + RIDAGEYR', data=da)
result = model.fit()
result.summary()

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:              MixedLM   Dependent Variable:   BPXDI1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                152.1061   
Min. group size:    106       Likelihood:           -20095.1838
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept            63.270    1.198 52.830 0.000 60.923 65.618
RIAGENDR[T.Male]      2.749    0.348  7.895 0.000  2.067  3.432
C(RIDRETH1)[T.2]      1.420    0.697  2.038 0.042  0.055  2.786
C(RIDRETH1)[T.3]      0.381    0.600  0.636 0.525 -0.794  1.556
C(RIDRETH1)[T.4]      2.413    0.659  3.662 0.000  1.122  3.704
C(RIDRETH1)[T.5]      3.317    0.690  4.804 0.000  1.964  4.670
RIDAGEYR             -0.032    0.015 -2.164 0.030 -0.061 -0.003
BMXBMI                0.195    0.026  7.587 0.000  0.145  0.246
group Var             9.731    0.379                           
group x RIDAGEYR Cov -0.142    0.007                           
RIDAGEYR Var          0.003    0.000                           
===============================================================

"""

__Q4b:__ Include a random intercept for age, and describe your findings.

The variance for our random intercept for age is very small, this lets me know that age does not have much bearing on the inter-cluster dependency that we observed in the previous question.